In [1]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import pandas as pd
import numpy as np
import tflearn
import tensorflow
import random

Using TensorFlow backend.


In [5]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
df = pd.read_excel('datset.xlsx')
df.head(5)

,Question,Answer,index
0,Can you ask me something about me?,"Nah, I'm good.",NaN
1,Can you ask me anything about myself?,"Nah, I'm good.",NaN
2,Can you ask me anything about me?,"Nah, I'm good.",NaN
3,Can you ask me anything?,"Nah, I'm good.",NaN
4,Can you ask me something?,"Nah, I'm good.",NaN


In [6]:
index = []
f = df["Answer"].drop_duplicates()
f["index1"] = f.index

In [9]:
from random import randrange
intents={}
t = []
n=f.count()
for i in range(n-2):
    question = []
    response=[]
    question.extend((df["Question"][f["index1"][i]:f["index1"][i+1]]))
    x = df["Answer"][f["index1"][i]]
    response.append((x))
    dic = {}
    dic["questions"] = question
    dic["responses"] = response
    dic["tag"] = i
    t.append(dic)
    intents["intents"] =t

In [10]:
for intent in intents['intents']:
    for question in intent['questions']:
        #tokenize each word
        w = nltk.word_tokenize(str(question))
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
nltk.download('wordnet')
words = sorted(list(set([lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words])))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between questions and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\P\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


9682 documents
88 classes [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87]
2469 unique lemmatized words ["'", "'d", "'ll", "'m", "'re", "'s", "'ve", ')', ',', '.', '1', '100', '2', '20', '3', '411', '5', '7', '9', ':', 'a', 'abandoned', 'able', 'abode', 'about', 'abroad', 'absolutely', 'abt', 'accounted', 'accurate', 'acknowledged', 'acquaintance', 'activist', 'activity', 'actor', 'actress', 'actual', 'actually', 'ad', 'address', 'adieu', 'adios', 'adorable', 'adult', 'adventure', 'aesthetically', 'af', 'affection', 'affirmative', 'afraid', 'african', 'after', 'afternoon', 'again', 'against', 'age', 'agender', 'agent', 'agnostic', 'ago', 'agree', 'ah', 'aha', 'ahead', 'ahh', 'ahoy', 'ai', 'air', '

In [12]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    question_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    question_words = [lemmatizer.lemmatize(word.lower()) for word in question_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in question_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])

In [13]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(np.array(train_x)[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(np.array(train_y)[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(np.array(train_x), np.array(train_y), n_epoch=1000, batch_size=8, show_metric=True)
model.save("model.tflearn")

print("model created")

Training Step: 1210999  | total loss: 0.05785 | time: 4.259s
| Adam | epoch: 1000 | loss: 0.05785 - acc: 0.9963 -- iter: 9680/9682
Training Step: 1211000  | total loss: 0.05207 | time: 4.262s
| Adam | epoch: 1000 | loss: 0.05207 - acc: 0.9967 -- iter: 9682/9682
--
INFO:tensorflow:C:\Users\P\Desktop\Projects\python-project-chatbot-codes\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
model created


In [123]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np
import random

In [130]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    resi = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(resi) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [131]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = str(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return (res)


In [141]:
#Creating GUI with tkinter
import tkinter
from tkinter import *
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))
        res = chatbot_response(msg)
        ChatLog.insert(END, "Kaori: " + str(res).strip('[' ']') + '\n\n')
        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)
base = Tk()
base.title("Kaori- A Talkative retrievel ChatBot ;>")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)
#Create Chat window
ChatLog = Text(base, bd=0, bg="#d2afff", height="8", width="50", font="Arial",)
ChatLog.config(state=DISABLED)
#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set
#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="10", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )
#Create the box to enter message
EntryBox = Text(base, bd=0, bg="#82a9c1",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)
#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=330, width=370)
EntryBox.place(x=6, y=350, height=90, width=370)
SendButton.place(x=6, y=450, height=40, width=370)
base.mainloop()